In [51]:
from langchain_groq import ChatGroq
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver

In [52]:
memory = MemorySaver()

In [9]:
llm = ChatGroq(model = "llama-3.3-70b-versatile", groq_api_key = "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")

In [10]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [12]:
tool = TavilySearchResults(max_results = 2, tavily_api_key = "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")

In [14]:
tool.invoke("What is langgraph?")

[{'title': 'LangGraph - Overview - Docs by LangChain',
  'url': 'https://docs.langchain.com/oss/python/langgraph/overview',
  'content': 'Trusted by companies shaping the future of agents - including Klarna, Replit, Elastic, and more - LangGraph is a low-level orchestration framework for building, managing, and deploying long-running, stateful agents.',
  'score': 0.9405774},
 {'title': 'What is LangGraph? - Analytics Vidhya',
  'url': 'https://www.analyticsvidhya.com/blog/2024/07/langgraph-revolutionizing-ai-agent/',
  'content': 'To sum up, LangGraph is a major advancement in the development of AI agents. It enables developers to push the limits of what’s possible with AI agents by eliminating the shortcomings of earlier systems and offering a flexible, graph-based framework for agent construction and execution. LangGraph is positioned to influence the direction of artificial intelligence significantly in the future. [...] LangGraph is a library built on top of Langchain that is desi

In [30]:
def multiply(a: int, b: int)->int:
    """
    Multiply a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: output int
    """
    return a * b

In [31]:
tools = [tool, multiply]

In [32]:
llm_with_tools = llm.bind_tools(tools)

In [33]:
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [54]:
builder_s = StateGraph(State)

In [55]:
builder_s.add_node("tool_calling_llm", tool_calling_llm)
builder_s.add_node("tools", ToolNode(tools))

builder_s.add_edge(START, "tool_calling_llm")
builder_s.add_conditional_edges("tool_calling_llm", tools_condition)
builder_s.add_edge("tools", "tool_calling_llm")

graph = builder_s.compile(checkpointer = memory)

In [58]:
config = {"configurable": {"thread_id": "1"}}

In [62]:
def get_response(query):
    response = graph.invoke({"messages": query}, config = config)
    return response["messages"][-1].content

In [65]:
print(get_response("Hi, my name is khademi"))

Hello Khademi, it's nice to meet you. I'll make sure to remember your name this time. Is there something I can help you with or would you like to chat?


In [66]:
print(get_response("What is my name?"))

Your name is Khademi.
